In [14]:
import numpy as np
import torch
import torch.nn as nn

best_model, char_to_int = torch.load("model_10.pth")
n_vocab = len(char_to_int)
int_to_char = dict((i, c) for c, i in char_to_int.items())

# Reload the model
class CharModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.lstm = nn.LSTM(input_size=1, hidden_size=256, num_layers=2, batch_first=True, dropout=0.2)
        self.dropout = nn.Dropout(0.2)
        self.linear = nn.Linear(256, n_vocab)
    def forward(self, x):
        x, _ = self.lstm(x)
        # take only the last output
        x = x[:, -1, :]
        # produce output
        x = self.linear(self.dropout(x))
        return x
    
model = CharModel()
model.load_state_dict(best_model)

# Randomly generate a prompt
# filename = "wonderland.txt"
# seq_length = 100
# raw_text = open(filename, 'r', encoding='utf-8').read()
# raw_text = raw_text.lower()
# start = np.random.randint(0, len(raw_text)-seq_length)
# prompt = raw_text[start:start+seq_length]
# pattern = [char_to_int[c] for c in prompt]

prompt = "CURRENT"
pattern = [char_to_int[c] for c in prompt]


model.eval()
print('Prompt: "%s"' % prompt)
with torch.no_grad():
    i = 0
    result = ''
    while i < 200 and result != '+':
        # Format input array of int into PyTorch tensor
        x = np.reshape(pattern, (1, len(pattern), 1)) / float(n_vocab)
        x = torch.tensor(x, dtype=torch.float32)
        
        # Generate logits as output from the model
        prediction = model(x)
        
        # Convert logits into one character
        index = int(prediction.argmax())
        result = int_to_char[index]
        print(result, end="")
        
        # Append the new character into the prompt for the next iteration
        pattern.append(index)
        pattern = pattern[1:]
        
        i += 1
        
print()
print("Done.")


Prompt: "CURRENT"
 AND TETHON AND THTH ANNMATIONS AND TETHON AND THTH ANNMATIONS AND TETHON AND THTH ANNMATIONS AND TETHON AND THTH ANNMATIONS AND TETHON AND THTH ANNMATIONS AND TETHON AND THTH ANNMATIONS AND TETHON AN
Done.
